In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/財工作業資料')

**put call parity : c + PV(X) = p + S**

In [ ]:
option = pd.read_csv("S&P 500 Option 0118-0215.csv")
index = pd.read_csv("S&P 500 Historical Data.csv")
futures = pd.read_csv("S&P 500 Futures Historical Data.csv")

index['Date'] = pd.to_datetime(index['Date'], format="%m/%d/%Y")
futures['Date'] = pd.to_datetime(futures['Date'], format="%m/%d/%Y")
option['date'] = pd.to_datetime(option['date'], format="%Y%m%d")
option['exdate'] = pd.to_datetime(option['exdate'], format="%Y%m%d")
option['last_date'] = pd.to_datetime(option['last_date'], format="%Y%m%d")
option['strike_price'] = option['strike_price']/1000  # 資料的有乘過1000
option['maturity_days'] = (option['exdate'] - option['date']).apply(lambda x: x.days)

settlement_type = 0 # am_settlement = 0 代表非上午結算
option = option[option['am_settlement'] == 0]
option

,secid,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,optionid,am_settlement,index_flag,issuer,exercise_style,maturity_days
6126,108105,2023-01-18,2023-01-18,2023-01-17,C,1000.0,2921.50,2945.5,151576116,0,1,CBOE S&P 500 INDEX,E,0
6127,108105,2023-01-18,2023-01-18,2023-01-17,C,1200.0,2721.50,2743.4,151576117,0,1,CBOE S&P 500 INDEX,E,0
6128,108105,2023-01-18,2023-01-18,NaT,C,1400.0,2521.50,2543.4,151576118,0,1,CBOE S&P 500 INDEX,E,0
6129,108105,2023-01-18,2023-01-18,NaT,C,1600.0,2321.50,2343.4,151576119,0,1,CBOE S&P 500 INDEX,E,0
6130,108105,2023-01-18,2023-01-18,NaT,C,1800.0,2121.50,2143.4,151576120,0,1,CBOE S&P 500 INDEX,E,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385551,108105,2023-02-15,2023-12-29,2023-02-09,P,5100.0,796.10,804.2,151950269,0,1,CBOE S&P 500 INDEX,E,317
385552,108105,2023-02-15,2023-12-29,2022-12-29,P,5200.0,885.10,893.5,151576481,0,1,CBOE S&P 500 INDEX,E,317
385553,108105,2023-02-15,2023-12-29,NaT,P,5300.0,976.30,984.9,152401344,0,1,CBOE S&P 500 INDEX,E,317
385554,108105,2023-02-15,2023-12-29,NaT,P,5400.0,1064.30,1082.7,151576482,0,1,CBOE S&P 500 INDEX,E,317


In [ ]:
class Call:
  def __init__(self, bid, offer, date, exdate, id, T, K):
    self.bid = bid
    self.offer = offer
    self.date = date.strftime('%Y-%m-%d')
    self.exdate = exdate.strftime('%Y-%m-%d')
    self.id = id
    self.T = T
    self.K = K
  def show(self):
    print(f"Call(date={self.date}, exdate={self.exdate}, id={self.id}, bid={self.bid}, offer={self.offer}, T={self.T}, K={self.K})")
  # def __repr__(self):
  #   return f"Put(bid={self.bid}, offer={self.offer}, date={self.date}, exdate={self.exdate}, id={self.id}, T={self.T}, K={self.K})"
class Put:
  def __init__(self, bid, offer, date, exdate, id, T, K):
    self.bid = bid
    self.offer = offer
    self.date = date.strftime('%Y-%m-%d')
    self.exdate = exdate.strftime('%Y-%m-%d')
    self.id = id
    self.T = T
    self.K = K
  def show(self):
    print(f"Put(date={self.date}, exdate={self.exdate}, id={self.id}, bid={self.bid}, offer={self.offer}, T={self.T}, K={self.K})")

In [ ]:
# r = 0.01844
r = 0.037 # risk free rate
transcation_cost = 1
arbitrage_opportunities = 0
price_all = option['strike_price'].unique()
price_all = np.sort(price_all)

for i in range(len(price_all)):
# for i in range(1):
  selected_strike_price = price_all[i]
  # selected_strike_price = 4000
  option_selected = option[(option['strike_price'] == selected_strike_price)] # 挑出所有K相同的option
  # when the option does not have the same strike price, option_selected == null
  while option_selected.empty != True:
    cp_flag = option_selected['cp_flag'].iloc[0]
    if cp_flag == 'C':
      bid = option_selected['best_bid'].iloc[0]
      offer = option_selected['best_offer'].iloc[0]
      date = option_selected['date'].iloc[0]
      exdate = option_selected['exdate'].iloc[0]
      id = option_selected['optionid'].iloc[0]
      T = option_selected['maturity_days'].iloc[0]/365
      K = option_selected['strike_price'].iloc[0]
      call = Call(bid, offer, date, exdate, id, T, K)

      put_option = option_selected[(option_selected['cp_flag']=='P')&(option_selected['date']==call.date)&(option_selected['exdate']==call.exdate)]
      offer = put_option['best_offer'].iloc[0]
      bid = put_option['best_bid'].iloc[0]
      id = put_option['optionid'].iloc[0]
      date = put_option['date'].iloc[0]
      exdate = put_option['exdate'].iloc[0]
      T = put_option['maturity_days'].iloc[0]/365
      K = put_option['strike_price'].iloc[0]
      put = Put(bid, offer, date, exdate, id, T, K)

    else:
      bid = option_selected['best_bid'].iloc[0]
      offer = option_selected['best_offer'].iloc[0]
      date = option_selected['date'].iloc[0]
      exdate = option_selected['exdate'].iloc[0]
      id = option_selected['optionid'].iloc[0]
      T = option_selected['maturity_days'].iloc[0]/360
      K = option_selected['strike_price'].iloc[0]
      put = Put(bid, offer, date, exdate, id, T, K)
      # 改成call
      call_option = option_selected[(option_selected['cp_flag']=='C')&(option_selected['date']==put.date)&(option_selected['exdate']==put.exdate)]
      offer = call_option['best_offer'].iloc[0]
      bid = call_option['best_bid'].iloc[0]
      id = call_option['optionid'].iloc[0]
      date = call_option['date'].iloc[0]
      exdate = call_option['exdate'].iloc[0]
      T = call_option['maturity_days'].iloc[0]/360
      K = call_option['strike_price'].iloc[0]
      call = Call(bid, offer, date, exdate, id, T, K)

    f_price = futures[futures['Date'] == call.date]
    futures_price = f_price['Price'].iloc[0]  # 該日的期貨價格
    i_price = index[index['Date'] == call.date]
    index_price = i_price['Price'].iloc[0]  # 該日的指數
    # verify put call parity
    if (call.bid + call.K*math.exp(-r*call.T)) - (put.offer + index_price) > 0: # if C > P
      # short call long put
      profit = (call.bid + call.K*math.exp(-r*call.T)) - (put.offer + futures_price)
      cost = transcation_cost * 3
      profit -= cost
      if profit > 0:
        print('1')
        print(f'Strategy: ')
        print(f'Short: call id= {call.id}, date= {call.date}, exdate= {call.exdate}, K= {call.K} and  borrow ${call.K*math.exp(-r*call.T)}')
        print(f'Long: put id= {put.id}, date= {put.date}, exdate= {put.exdate}, K= {put.K} and futures ${futures_price}')
        print(f'Profit = {profit}')
        print('------------------------')
        arbitrage_opportunities += 1
    if (put.bid + index_price) - (call.offer + call.K*math.exp(-r*call.T)) > 0: # if C < P
      # short put long call
      profit = (put.bid + futures_price) - (call.offer + call.K*math.exp(-r*call.T))
      cost = transcation_cost * 3
      profit -= cost
      if profit > 0:
        print('2')
        print(f'Strategy: ')
        print(f'Short: put id= {put.id}, date= {put.date}, exdate= {put.exdate}, K= {put.K} and futures ${futures_price}')
        print(f'Long: call id= {call.id}, date= {call.date}, exdate= {call.exdate}, K= {call.K} and  save ${call.K*math.exp(-r*call.T)}')
        print(f'Profit = {profit}')
        print('------------------------')
        arbitrage_opportunities += 1

    option_selected = option_selected.drop(option[(option['date'] == call.date) & (option['optionid'] == call.id)].index)
    option_selected = option_selected.drop(option[(option['date'] == put.date) & (option['optionid'] == put.id)].index)

print('Total: arbitrage_opportunities', arbitrage_opportunities)

串流輸出內容已截斷至最後 5000 行。
Profit = 33.298031516828814
------------------------
2
Strategy: 
Short: put id= 151576421, date= 2023-02-02, exdate= 2023-12-29, K= 1600.0 and futures $4191.5
Long: call id= 151576355, date= 2023-02-02, exdate= 2023-12-29, K= 1600.0 and  save $1547.3620447829678
Profit = 34.537955217032504
------------------------
2
Strategy: 
Short: put id= 151713783, date= 2023-02-03, exdate= 2023-02-24, K= 1600.0 and futures $4147.75
Long: call id= 151713707, date= 2023-02-03, exdate= 2023-02-24, K= 1600.0 and  save $1596.5975953511934
Profit = 8.952404648806805
------------------------
2
Strategy: 
Short: put id= 149407848, date= 2023-02-03, exdate= 2023-02-28, K= 1600.0 and futures $4147.75
Long: call id= 149407720, date= 2023-02-03, exdate= 2023-02-28, K= 1600.0 and  save $1595.9503390665081
Profit = 9.799660933491396
------------------------
2
Strategy: 
Short: put id= 151914845, date= 2023-02-03, exdate= 2023-03-03, K= 1600.0 and futures $4147.75
Long: call id= 151914768, 

KeyboardInterrupt: ignored